# ID3 Decision tree algorithm
Another approach to decision tree learning is the ID3 algorithm. ID3 is a top-down, greedy algorithm that builds a decision tree by choosing the attribute that best classifies the training examples at each step. The ID3 algorithm is a special case of the more general C4.5 algorithm, which is a modification of ID3 that can handle continuous attributes. The ID3 algorithm is described in detail in Quinlan (1986). The ID3 algorithm is summarized below.

#### Import libraries

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

#### Load dataset
As ID3 algorithm can only handle categorical data, we will use the approbriate dataset [play_tennis.csv](play_tennis.csv) for this algorithm. 

In [2]:
dataset = pd.read_csv('play_tennis.csv', index_col=0)
y = dataset['play']
X = dataset.drop('play', axis=1)

In [3]:
y = y.to_numpy()
y

array(['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes',
       'Yes', 'Yes', 'Yes', 'No'], dtype=object)

In [4]:
X = X.to_numpy()
X

array([['Sunny', 'Hot', 'High', 'Weak'],
       ['Sunny', 'Hot', 'High', 'Strong'],
       ['Overcast', 'Hot', 'High', 'Weak'],
       ['Rain', 'Mild', 'High', 'Weak'],
       ['Rain', 'Cool', 'Normal', 'Weak'],
       ['Rain', 'Cool', 'Normal', 'Strong'],
       ['Overcast', 'Cool', 'Normal', 'Strong'],
       ['Sunny', 'Mild', 'High', 'Weak'],
       ['Sunny', 'Cool', 'Normal', 'Weak'],
       ['Rain', 'Mild', 'Normal', 'Weak'],
       ['Sunny', 'Mild', 'Normal', 'Strong'],
       ['Overcast', 'Mild', 'High', 'Strong'],
       ['Overcast', 'Hot', 'Normal', 'Weak'],
       ['Rain', 'Mild', 'High', 'Strong']], dtype=object)

## ID3 Algorithm code

Let's create node class for using in ID3 algorithm.

In [5]:
class Node:
    def __init__(self, decision: int = None, attribute: int = None) -> None:
        """
        Create a node in the decision tree.
        """
        self.decision = decision
        self.attribute = attribute
        self.children = {}

And this is the main class of ID3 algorithm.
He uses this hyperparameters:
* *max_depth*: maximum depth of the tree
* *min_samples_split*: minimum number of samples required to split an internal node

In [6]:
class ID3DecisionTree:
    def __init__(self, max_depth: int = 3, min_samples_split: int = 0):
        """
        Initialize the decision tree.

        Parameters
        ----------
        max_depth : int
            The maximum depth of the tree.
        min_samples_split : int
            The minimum number of samples required to split an internal node.
        """
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def _entropy(self, y: np.array) -> float:
        """
        Returns the entropy of a given set of labels.
        """
        _, counts = np.unique(y, return_counts=True)
        p = counts / len(y)
        return -np.sum(p * np.log2(p))
    
    def _information_gain(self, X: np.array, y: np.array, attribute: int) -> float:
        """
        Returns the information gain of a given attribute.
        """
        unique, counts = np.unique(X[:, attribute], return_counts=True)
        p = counts / len(X)
        entropy = np.sum(p * self._entropy(y[X[:, attribute] == unique[0]]))
        return self._entropy(y) - entropy
    
    def _best_attribute(self, X: np.array, y: np.array, attributes: np.array) -> tuple:
        """
        Returns the best attribute to split on and its index in the attributes array.
        """
        a = [self._information_gain(X, y, attribute) for attribute in attributes]
        return (np.argmax(a), attributes[np.argmax(a)])
    
    def _major_class(self, y: np.array) -> int:
        """
        Returns the most common class in a given set of labels.
        """
        unique, counts = np.unique(y, return_counts=True)
        return unique[np.argmax(counts)]

    def fit(self, X, y) -> None:
        """
        Build decision tree classifier.
        """
        features = np.arange(X.shape[1])
        self.tree = self._build_tree(X, y, features)

    def _build_tree(self, X: np.array, y: np.array, features: np.array, depth: int = 0) -> Node:
        """
        Recursively build the decision tree.
        """
        if np.unique(y).size == 1:
            return Node(y[0])
        
        if features.size == 0:
            return Node(self._major_class(y))
        
        if depth >= self.max_depth or X.size < self.min_samples_split:
            return Node(self._major_class(y))
        
        best_attr = self._best_attribute(X, y, features)
        node = Node(None, best_attr[1])

        for value in np.unique(X[:, best_attr]):
            subset_X = X[X[:, best_attr[1]] == value]
            subset_y = y[X[:, best_attr[1]] == value]

            if subset_X.size > 0:
                node.children[value] = self._build_tree(subset_X, subset_y, np.delete(features, best_attr[0]), depth=depth+1)
            else:
                node.children[value] = Node(self._major_class(y))

        return node
    
    def _predict(self, x: np.array, node: Node) -> int:
        """
        Recursively predict the class of a given sample.
        """
        if node.decision is not None:
            return node.decision
        else:
            return self._predict(x, node.children[x[node.attribute]])

    def predict(self, X: np.array) -> np.array:
        """
        Predict classes for X samples.
        """
        return np.array([self._predict(x, self.tree) for x in X])
    
    def evaluate(self, X: np.array, y: np.array) -> float:
        """
        Evaluate the accuracy of the decision tree.
        """
        return np.sum(self.predict(X) == y) / len(y)
    
  

### Test ID3 algorithm.
We create a tree with maximum depth of 3 and minimum number of samples required to split an internal node is 2.
Interface of the tree is similar to sklearn decision tree classifier.

In [7]:
clf = ID3DecisionTree(max_depth=3, min_samples_split=2)
clf.fit(X, y)
clf.evaluate(X, y)

1.0

As we can see, the accuracy of the tree is 1.0, which means that the tree perfectly classifies the training set. Of course, the tree is overfitting the training set. So, let's try to split out dataset into train and test sets and check the accuracy of the tree on the test set.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
clf2 = ID3DecisionTree(max_depth=3, min_samples_split=2)

clf2.fit(X_train, y_train)
clf2.evaluate(X_test, y_test)

1.0